In [8]:
import h5py
import numpy
import tensorflow as tf
import random
import json
import spacy
import nltk
from nltk.stem.porter import *
from nltk.stem.lancaster import LancasterStemmer
class Chatbot:

    showWorking = False
    train_again=False
    vocabulary=[]
    labels=[]
    responseDictionary = {}
    stemmer = PorterStemmer()
    labelFileName= 'LabelList.json'
    vocabularyFileName = 'VocabularyList.json'
    responseFileName = 'responseDictionary.json'
    dataSetFileName = 'CommonIntentions.json'
    feedbackFileName = 'feedback.json'
    modelFileName = 'chatBotContext.h5'
    
    def __init__(self,train_again=False):
        Chatbot.train_again = train_again

    def get_model(self):

        if not Chatbot.train_again:
            with open (Chatbot.labelFileName) as fl:
                Chatbot.labels = json.load(fl)
            
            with open (Chatbot.vocabularyFileName) as fv:
                Chatbot.vocabulary = json.load(fv)
                
            with open (responseFileName) as fr:
                Chatbot.responseDictionary = json.load(fr)
                
            model= tf.keras.models.load_model(Chatbot.modelFileName)
        else:
            with open(Chatbot.dataSetFileName) as file:
                data = json.load(file)

            docs_x = []
            docs_y = []

            for intent in data['intents']:
                
                Chatbot.responseDictionary[intent['tag']] = []
                responseList=[]
                for response in intent['responses']:
                    responseList.append(response)
                    
                Chatbot.responseDictionary[intent['tag']] = responseList
                
                for pattern in intent['patterns']:
                    wrds = remove_stop_words_get_lemmas(pattern)
                    if len(wrds)>0:
                        Chatbot.vocabulary.extend(wrds)
                        docs_x.append(wrds)
                        docs_y.append(intent['tag'])               

                if intent['tag'] not in Chatbot.labels:
                    Chatbot.labels.append(intent['tag'])
                    
            with open(Chatbot.responseFileName, 'w') as fp:
                json.dump(Chatbot.responseDictionary, fp)
                
            Chatbot.vocabulary = sorted(list(set(Chatbot.vocabulary)))
            Chatbot.labels = sorted(Chatbot.labels)
            training = []
            output = []
            out_empty = [0 for _ in range(len(Chatbot.labels))]

            for x, doc in enumerate(docs_x):
                bag = []

                for w in Chatbot.vocabulary:
                    if w in doc:
                        bag.append(1)
                    else:
                        bag.append(0)

                output_row = out_empty[:]
                output_row[Chatbot.labels.index(docs_y[x])] = 1

                training.append(bag)
                output.append(output_row)

            training = numpy.array(training)
            output = numpy.array(output)


            model = tf.keras.Sequential()
            for layer in range(3):
            # Adds a densely-connected layer with specified units to the model:
                model.add(tf.keras.layers.Dense(10, activation='relu'))

        # Add a sigmoid layer with 1 output units:
            model.add(tf.keras.layers.Dense(len(output[0]), activation='softmax'))
            model.compile(optimizer='adam',loss='categorical_crossentropy',metric=['accuracy'])
            model.fit(training, output, epochs=300)
            model.save(Chatbot.modelFileName)
            
            with open(Chatbot.labelFileName, 'w') as fl:
                json.dump(Chatbot.labels, fl)
            with open(Chatbot.vocabularyFileName, 'w') as fv:
                json.dump(Chatbot.vocabulary, fv)
        return model

    def get_yes_list(self):
        return ['yes','sure','ok','okay','for sure','sure','totally','yep','ya','yeah','yup','certainly',
                         'definitely','of course','gladly','absolutely','indeed']
    @staticmethod
    def feedback_save_to_file(sentence,PredictedTag):
        feedbackDict = {}
        feedbackDict['input'] = sentence
        feedbackDict['PredictedTag']=PredictedTag
        
        with open(chatbot.feedbackFileName, 'a+') as fv:
                json.dump(feedbackDict, fv)
        
    
    def chat(self):
        
        print("Do you want to see internal processing steps:")
        choice = input("Enter you choice : ")
        if choice.lower() in self.get_yes_list():
            Chatbot.showWorking=True
        model = self.get_model()
        print("Start talking with the bot (type quit to stop)!")
        while True:
            inp = input("You: ")
            if inp.lower() == "quit":
                break
            results = model.predict(bag_of_words(inp))
            results_index = numpy.argmax(results)
            tag = Chatbot.labels[results_index]
            Probability = results[0][results_index]
            if Chatbot.showWorking:
                print("Highest intent proability:",Probability)
                print("Intent of sentence : ",tag)
            responses = Chatbot.responseDictionary[tag]
            
            if Probability < .7:
                Chatbot.feedback_save_to_file(inp,tag)
                print('***** I am not able to get that clearly best possible response I can give is *****')
                
            print("chatbot:",random.choice(responses))
            
def remove_stop_words_get_lemmas(sentence):
        s_words = nltk.word_tokenize(sentence)
        s_words = [Chatbot.stemmer.stem(word.lower()) for word in s_words]
        if Chatbot.showWorking:
            print('Stems :',s_words)
        return s_words

def bag_of_words(sentence):
    bags = []
    
    #creating hot vector
    bag = [0 for _ in range(len(Chatbot.vocabulary))] 
    s_words = remove_stop_words_get_lemmas(sentence)

    for se in s_words:
        for i, w in enumerate(Chatbot.vocabulary):
            if w == se:
                bag[i] = 1
                
    if Chatbot.showWorking:
        print('Vector Representation of input sentence is:',bag)
            
    bags.append(bag)
    bags = numpy.array(bags)
    return bags
        
def main():
    chatbot = Chatbot(False)
    chatbot.chat()

if __name__ == '__main__':
    main()
    


Do you want to see internal processing steps:
Enter you choice : yes
Stems : ['tell', 'me', 'some', 'car', 'which', 'are', 'in', 'my', 'budget']
Stems : ['cheap', 'car']
Stems : ['cheapest', 'car']
Stems : ['afford', 'car']
Stems : ['low', 'budget', 'car']
Stems : ['lower', 'end', 'car']
Stems : ['low', 'maintainc', 'car']
Stems : ['low', 'cost', 'car']
Stems : ['econom', 'car']
Stems : ['which', 'car', 'is', 'slowest', '?']
Stems : ['which', 'is', 'slowest', 'car', '?']
Stems : ['slow', 'car']
Stems : ['afford', 'luxuri', 'car']
Stems : ['top', 'car']
Stems : ['high', 'class', 'car']
Stems : ['high', 'end', 'car']
Stems : ['expens', 'car']
Stems : ['which', 'car', 'is', 'fastest', '?']
Stems : ['which', 'is', 'fastest', 'car']
Stems : ['fast', 'car']
Stems : ['what', 'car', 'i', 'should', 'buy']
Stems : ['which', 'car', 'to', 'buy']
Stems : ['trend', 'car']
Stems : ['differ', 'compani', 'of', 'car']
Stems : ['which', 'is', 'the', 'latest', 'car']
Stems : ['tell', 'me', 'some', 'mileag

Stems : ['hey', 'i', 'am']
Stems : ['your', 'name', '?']
Stems : ['who', 'are', 'you', '?']
Stems : ['where', 'to', 'get', 'inform', 'about', 'chatbot']
Stems : ['what', 'is', 'chatbot']
Stems : ['defin', 'chatbot']
Stems : ['creat', 'chatbot']
Stems : ['who', 'invent', 'chatbot']
Stems : ['first', 'chatbot', 'come', 'in']
Stems : ['chatbot']
Stems : ['chat']
Stems : ['bot']
Stems : ['chatbot', 'exit', 'sinc']
Stems : ['name', 'of', 'first', 'chatbot']
Stems : ['histori', 'of', 'chatbot']
Stems : ['use', 'of', 'chatbot']
Stems : ['advantag', 'of', 'chatbot']
Stems : ['applic', 'of', 'chatbot']
Stems : ['pratic', 'use', 'of', 'chatbot']
Stems : ['whi', 'should', 'i', 'use', 'you']
Stems : ['whi', 'to', 'use', 'you']
Stems : ['what', 'i', 'will', 'get', 'out', 'of']
Stems : ['pro', 'of', 'chatbot']
Stems : ['tell', 'me', 'someth', 'bad', 'about', 'you']
Stems : ['case', 'where', 'chatbot', 'fail']
Stems : ['chatbot']
Stems : ['see', 'you', 'later']
Stems : ['bye']
Stems : ['see', 'you', 

Epoch 24/300
341/341 [==============================] - ETA: 0s - loss: 2.719 - 0s 132us/sample - loss: 2.8036
Epoch 25/300
341/341 [==============================] - ETA: 0s - loss: 2.729 - 0s 109us/sample - loss: 2.7365
Epoch 26/300
341/341 [==============================] - ETA: 0s - loss: 2.827 - 0s 132us/sample - loss: 2.6708
Epoch 27/300
341/341 [==============================] - ETA: 0s - loss: 2.527 - 0s 138us/sample - loss: 2.6077
Epoch 28/300
341/341 [==============================] - ETA: 0s - loss: 2.476 - 0s 129us/sample - loss: 2.5444
Epoch 29/300
341/341 [==============================] - ETA: 0s - loss: 2.389 - 0s 150us/sample - loss: 2.4822
Epoch 30/300
341/341 [==============================] - ETA: 0s - loss: 2.238 - 0s 161us/sample - loss: 2.4204
Epoch 31/300
341/341 [==============================] - ETA: 0s - loss: 2.249 - ETA: 0s - loss: 2.372 - 0s 179us/sample - loss: 2.3594
Epoch 32/300
341/341 [==============================] - ETA: 0s - loss: 2.121 - 0s 120us

Epoch 98/300
341/341 [==============================] - ETA: 0s - loss: 0.239 - 0s 117us/sample - loss: 0.2399
Epoch 99/300
341/341 [==============================] - ETA: 0s - loss: 0.407 - 0s 106us/sample - loss: 0.2336
Epoch 100/300
341/341 [==============================] - ETA: 0s - loss: 0.134 - 0s 111us/sample - loss: 0.2275
Epoch 101/300
341/341 [==============================] - ETA: 0s - loss: 0.216 - 0s 109us/sample - loss: 0.2208
Epoch 102/300
341/341 [==============================] - ETA: 0s - loss: 0.209 - 0s 106us/sample - loss: 0.2169
Epoch 103/300
341/341 [==============================] - ETA: 0s - loss: 0.271 - 0s 109us/sample - loss: 0.2108
Epoch 104/300
341/341 [==============================] - ETA: 0s - loss: 0.114 - 0s 126us/sample - loss: 0.2059
Epoch 105/300
341/341 [==============================] - ETA: 0s - loss: 0.172 - 0s 114us/sample - loss: 0.2015
Epoch 106/300
341/341 [==============================] - ETA: 0s - loss: 0.214 - 0s 114us/sample - loss: 0

Epoch 171/300
341/341 [==============================] - ETA: 0s - loss: 0.055 - 0s 66us/sample - loss: 0.0682
Epoch 172/300
341/341 [==============================] - ETA: 0s - loss: 0.102 - 0s 66us/sample - loss: 0.0663
Epoch 173/300
341/341 [==============================] - ETA: 0s - loss: 0.049 - 0s 66us/sample - loss: 0.0679
Epoch 174/300
341/341 [==============================] - ETA: 0s - loss: 0.022 - 0s 66us/sample - loss: 0.0646
Epoch 175/300
341/341 [==============================] - ETA: 0s - loss: 0.054 - 0s 73us/sample - loss: 0.0660
Epoch 176/300
341/341 [==============================] - ETA: 0s - loss: 0.029 - 0s 66us/sample - loss: 0.0640
Epoch 177/300
341/341 [==============================] - ETA: 0s - loss: 0.107 - 0s 81us/sample - loss: 0.0653
Epoch 178/300
341/341 [==============================] - ETA: 0s - loss: 0.098 - 0s 73us/sample - loss: 0.0634
Epoch 179/300
341/341 [==============================] - ETA: 0s - loss: 0.046 - 0s 66us/sample - loss: 0.0623
E

341/341 [==============================] - ETA: 0s - loss: 0.064 - 0s 67us/sample - loss: 0.0394
Epoch 245/300
341/341 [==============================] - ETA: 0s - loss: 0.033 - 0s 67us/sample - loss: 0.0362
Epoch 246/300
341/341 [==============================] - ETA: 0s - loss: 0.032 - 0s 67us/sample - loss: 0.0367
Epoch 247/300
341/341 [==============================] - ETA: 0s - loss: 0.030 - 0s 67us/sample - loss: 0.0359
Epoch 248/300
341/341 [==============================] - ETA: 0s - loss: 0.087 - 0s 70us/sample - loss: 0.0366
Epoch 249/300
341/341 [==============================] - ETA: 0s - loss: 0.013 - 0s 70us/sample - loss: 0.0362
Epoch 250/300
341/341 [==============================] - ETA: 0s - loss: 0.032 - 0s 132us/sample - loss: 0.0353
Epoch 251/300
341/341 [==============================] - ETA: 0s - loss: 0.012 - 0s 147us/sample - loss: 0.0356
Epoch 252/300
341/341 [==============================] - ETA: 0s - loss: 0.040 - ETA: 0s - loss: 0.037 - 0s 217us/sample - l

Vector Representation of input sentence is: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0

NameError: name 'chatbot' is not defined

In [ ]:
import spacy
from importlib import reload 
reload(spacy)
nlp = spacy.load('en_core_web_sm')

doc = nlp(u"Apples and oranges are similar. Boots and hippos aren't.")
for token in doc:
    print(token, token.lemma_)

In [ ]:
import spacy

while (True):

    doc = input("Enter :")

    if doc=='quit':
        break
    spacy_nlp = spacy.load('en_core_web_sm')
    spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
    
    doc = spacy_nlp(doc)
    customize_stop_words = [
        '.', '?','!'
    ]
    
    for w in customize_stop_words:
        spacy_nlp.vocab[w].is_stop = True

    tokens = [token.lemma_ for token in doc if not token.is_stop]

    print(tokens)




In [ ]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')
print(spacy.lang.en.stop_words.STOP_WORDS)

In [ ]:
def bag_of_words(s, words):
    bags = []
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    #print('tokenizer:',s_words)
    
    print('lemmarizer:',get_lemmas(s))
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    bags.append(bag)
    bags = numpy.array(bags)
    #return numpy.array(bag)
    return bags

In [ ]:
import pickle

    
with open ('responseDictionary', 'rb') as fp:
    itemlist2 = pickle.load(fp)
    
print(itemlist2.keys())

In [ ]:
import nltk 
#from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import *
stemmer = PorterStemmer()
sentence = 'what are you'
s_words = nltk.word_tokenize(sentence)
s_words = [stemmer.stem(word.lower()) for word in s_words]

print(s_words)


    


In [ ]:
dict = {'asds': 'Geeks', 'b': 'For', 'c': 'geeks'} 

def getList(dict): 
      
    return [*dict] 

print(dict['dhjjh']) 